# Operator Algebra Logic

At the heart of Pyxu's microservice architecture is a powerful operator algebra logic allowing to create versatile and
complex operators/functionals from fundamental building blocks. In this section, we'll guide you through how to create
and manipulate operators to construct intricate functionals and mappings. By the end, you'll be an expert in leveraging
the power of Pyxu's operator algebra!

## Arithmetic Operations on Operators

In Pyxu, you can use a variety of arithmetic operations to combine basic operators into more complex ones. Here are some
simple but powerful commands you can use:

```python
>> op1 + op2  # Addition of two operators
>> op1 * op2  # Composition of two operators
>> op ** 3    # Exponentiation of an operator
>> op.argscale(c)  # Dilation by a scalar 'c'
>> op.argshift(x)  # Shifting by a vector 'x'
>> 4 * op  # Scaling by a scalar
>> op.T  # Transposing
```

### How Does it Work? 🛠️

Every time you perform an arithmetic operation, Pyxu automatically infers the output type based on the properties of the
operators involved in the operation. This type inference is super convenient because it saves you from manual
calculations! 

For example, Pyxu takes care of updating as needed methods like `apply()`[🔗](../api/abc.html#pyxu.abc.Map.apply),
`jacobian()`[🔗](../api/abc.html#pyxu.abc.DiffMap.jacobian), `grad()`[🔗](../api/abc.html#pyxu.abc.DiffFunc.grad),
`prox()`[🔗](../api/abc.html#pyxu.abc.ProxFunc.prox), and `adjoint()`[🔗](../api/abc.html#pyxu.abc.LinOp.adjoint)
according to arithmetic rules. This means you can plug these composite operators directly into proximal gradient
algorithms without havng to implement manually gradients or proximal steps yourself.

## Behind the Scenes: Arithmetic Rules

For those who love to peek under the hood, Pyxu utilizes a set of arithmetic rules located in the
`pyxu.abc.arithmetic`[🔗](../api/abc/arithmetic.html) module:

- `Rule`[🔗](../api/abc.html#pyxu.abc.arithmetic.Rule): Base class for all arithmetic rules.
- `ScaleRule`[🔗](../api/abc.html#pyxu.abc.arithmetic.ScaleRule): Handles scaling of operators by scalars.
- `ArgScaleRule`[🔗](../api/abc.html#pyxu.abc.arithmetic.ArgScaleRule): Manages the dilation of operator parameters.
- `ArgShiftRule`[🔗](../api/abc.html#pyxu.abc.arithmetic.ArgShiftRule): Takes care of shifting operator parameters.
- `AddRule`[🔗](../api/abc.html#pyxu.abc.arithmetic.AddRule): Manages the addition of two operators.
- `ChainRule`[🔗](../api/abc.html#pyxu.abc.arithmetic.ChainRule): Deals with the composition of two operators.
- `PowerRule`[🔗](../api/abc.html#pyxu.abc.arithmetic.PowerRule): Handles operator exponentiation..
- `TransposeRule`[🔗](../api/abc.html#pyxu.abc.arithmetic.TransposeRule): Takes care of transposing a linear operator.

### Working Example 🎯

Consider the composition of a `DiffFunc`[🔗](../api/abc.html#pyxu.abc.DiffFunc) with a
`DiffMap`[🔗](../api/abc.html#pyxu.abc.DiffMap). Let `f` be a `DiffFunc` and `L` be a `DiffMap`. Then their composition
`h` is another `DiffFunc`, and the gradient is given by:

```python
>> h = f * L
>> h.grad(x) = L.jacobian(x).adjoint(f.grad(L(x)))
```

More generally, here is how `ChainRule` updates the various core methods when a composition of the form `lhs * rhs` is
performed:

- `apply()` and `lipschitz`:
    
    ```python
    op.apply(arr) = lhs.apply(rhs.apply(arr))
    op.lipschitz = lhs.lipschitz * rhs.lipschitz
    ```

- `prox()`:

    ```python
    op.prox(arr, tau) = rhs.adjoint(lhs.prox(rhs.apply(arr), tau))
    ```

- `jacobian()` and `diff_lipschitz`:

    ```python
    op.jacobian(arr) = lhs.jacobian(rhs.apply(arr)) * rhs.jacobian(arr)
    op.diff_lipschitz =
        linear * linear  => 0
        linear * diff    => lhs.lipschitz * rhs.diff_lipschitz
        diff   * linear  => lhs.diff_lipschitz * (rhs.lipschitz ** 2)
        diff   * diff    => infty
    ```

- `grad()`:

    ```python
    op.grad(arr) = lhs.grad(rhs.apply(arr)) @ rhs.jacobian(arr).asarray()
    ```

## Building Block-Operators 

You can even define block-operators using `coo_block`[🔗](../api/operator/blocks.html#pyxu.operator.coo_block).
Alternatively, higher-level functions like 
`block`[🔗](../api/operator/blocks.html#pyxu.operator.block),
`block_diag`[🔗](../api/operator/blocks.html#pyxu.operator.block_diag),
`stack`[🔗](../api/operator/blocks.html#pyxu.operator.stack),
`vstack`[🔗](../api/operator/blocks.html#pyxu.operator.vstack), and
`hstack`[🔗](../api/operator/blocks.html#pyxu.operator.hstack) can also be used.

For example, the code below: 

```python
>> coo_block(([A(500,1000), B(1,1000), C(500,500), D(1,3)],  # data
   ...      [[0, 1, 0, 2],  # i
   ...       [0, 0, 2, 1],  # j
            ]),grid_shape=(3, 3))
```

results in a block sparse composite operator of the form:

| coarse_idx |      0       |    1    |      2      |
|------------|--------------|---------|-------------|
|          0 | A(500, 1000) |         | C(500, 500) |
|          1 | B(1, 1000)   |         |             |
|          2 |              | D(1, 3) |             |


Similarly, a functional $h(x) = \sum_{i=1}^{3} f_i(K_ix)$, can be constructed as follows:

```python
f = hstack([f_1, f_2, f_3]) * vstack([K_1, K_2, K_3])
```

Again, Pyxu takes care of all the heavy lifting by automatically inferring the output type and arithmetic methods of the
block-operator from its building blocks.